In [ ]:
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.functions import col, explode, collect_list, concat_ws, when, lit, to_date, size, first, isnan
from pyspark.sql.types import FloatType
from pathlib import Path
import logging
import json
import sys
sys.path.append(str(Path('../scripts')))
from config import MOVIE_IDS,TMDB_API_KEY,BASE_URL,RAW_DATA_DIR, PROCESSED_DATA_DIR
from api_fetch import initialize_spark, create_movie_dataframe
from data_cleaning import clean_data
from pyspark.sql import functions as F
from pyspark.sql.types import StringType, IntegerType

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

In [ ]:
spark = initialize_spark()
raw_df = create_movie_dataframe(MOVIE_IDS, TMDB_API_KEY, BASE_URL, RAW_DATA_DIR, spark)

2025-04-20 21:18:09,616 - INFO - Spark session initialized with custom configurations
2025-04-20 21:18:09,697 - INFO - Loaded cached data from data\raw\raw_movies_20250420_210921.json
2025-04-20 21:18:13,489 - WARNING - Attempt 1 failed for movie ID 0: 404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/0?api_key=98eb78059acf277a9397adb9a104b869&append_to_response=credits
2025-04-20 21:18:15,586 - WARNING - Attempt 2 failed for movie ID 0: 404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/0?api_key=98eb78059acf277a9397adb9a104b869&append_to_response=credits
2025-04-20 21:18:18,777 - WARNING - Attempt 3 failed for movie ID 0: 404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/0?api_key=98eb78059acf277a9397adb9a104b869&append_to_response=credits
2025-04-20 21:18:18,778 - ERROR - Max retries reached for movie ID 0
2025-04-20 21:18:18,781 - WARNING - Skipping movie ID 0 due to fetch failure
2025-04-20 21:18:18,782 - INFO - Usin

----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 7587)
Traceback (most recent call last):
  File "C:\Users\skele\AppData\Local\Programs\Python\Python311\Lib\socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "C:\Users\skele\AppData\Local\Programs\Python\Python311\Lib\socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "C:\Users\skele\AppData\Local\Programs\Python\Python311\Lib\socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "C:\Users\skele\AppData\Local\Programs\Python\Python311\Lib\socketserver.py", line 755, in __init__
    self.handle()
  File "d:\Downloads\www.amalitech.org\Projects\my_de_projects\spark_tmdb_analysis\spark_venv\Lib\site-packages\pyspark\accumulators.py", line 295, in handle
    poll(accum_updates)
  File "d:\Downloads\www.amalitech.o

In [ ]:
raw_df.show()

+-----+--------------------+---------------------+---------+--------------------+--------------------+------+---------+--------------+-----------------+-----------------+--------------------+----------+--------------------+--------------------+--------------------+------------+----------+-------+--------------------+--------+------------------+-----------------+-----+------------+----------+--------------------+
|adult|       backdrop_path|belongs_to_collection|   budget|              genres|            homepage|    id|  imdb_id|origin_country|original_language|   original_title|            overview|popularity|         poster_path|production_companies|production_countries|release_date|   revenue|runtime|    spoken_languages|  status|           tagline|            title|video|vote_average|vote_count|             credits|
+-----+--------------------+---------------------+---------+--------------------+--------------------+------+---------+--------------+-----------------+----------------

In [7]:
raw_df = (raw_df
    # Drop columns
    .drop(*columns_to_drop)
    
    # Process JSON columns
    .withColumn("belongs_to_collection", 
        F.when(F.col("belongs_to_collection").isNotNull(), 
              F.col("belongs_to_collection.name")))
    
    .withColumn("genres", 
        F.array_join(F.transform(F.col("genres"), 
                               lambda x: x["name"]), " | "))
    
    .withColumn("spoken_languages", 
        F.array_join(F.transform(F.col("spoken_languages"), 
                               lambda x: x["iso_639_1"]), " | "))
    
    .withColumn("production_countries", 
        F.array_join(F.transform(F.col("production_countries"), 
                               lambda x: x["iso_3166_1"]), " | "))
    
    .withColumn("production_companies", 
        F.array_join(F.transform(F.col("production_companies"), 
                               lambda x: x["name"]), " | "))
    
    .withColumn("origin_country", 
        F.array_join(F.array_sort(F.col("origin_country")), " | "))
    
    # Process credits
    .withColumn("cast", 
        F.array_join(F.transform(F.col("credits.cast"), 
                               lambda x: x["name"]), ", "))
    
    .withColumn("cast_size", 
        F.size(F.col("credits.cast")))
    
    .withColumn("director", 
        F.array_join(
            F.filter(F.col("credits.crew"), 
                    lambda x: x["job"] == "Director")["name"], 
            ", "))
    
    .withColumn("crew_size", 
        F.size(F.col("credits.crew")))
    
    # Drop original credits
    .drop("credits")
)